In [ ]:
# ====== RADIAL TOKENIZER (SIMULATED PROJECTION) TEST ======
import torch
import torch.nn as nn
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from radial_tokenizer import RadialTokenizer

# ==== CONFIG ====
IMG_PATH = "C:/Users/denni/Downloads/test.png"
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"

# Create directories for the new outputs
TOK9_DIR = os.path.join(OUTPUT_DIR, "tokens_9D")
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D_projected") # For projected tokens
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights_test") # For test weights

os.makedirs(TOK9_DIR, exist_ok=True)
os.makedirs(TOK192_DIR, exist_ok=True)
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# ==== LOAD IMAGE ====
image = cv2.imread(IMG_PATH)
if image is None:
    raise FileNotFoundError(f"Image not found at {IMG_PATH}")

image = cv2.resize(image, (128, 128))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_norm = image_rgb / 255.0
img_tensor = torch.from_numpy(image_norm).permute(2, 0, 1).unsqueeze(0).float()
print(f"✅ Loaded Image Tensor Shape: {img_tensor.shape}")

# ==== 1. TEST RADIAL TOKENIZER (9D Output) ====
tokenizer = RadialTokenizer()
tokens_9d = tokenizer(img_tensor)  # Shape: [1, 4, 9]
print(f"✅ Raw Tokens Shape (9D): {tokens_9d.shape}")

# ==== 2. SIMULATE THE MODEL'S PROJECTION LAYER ====
# This step mimics what ModifiedMobileViT does internally.
# We do it here to get the 192D tokens needed for the positional encoding test.
projection_layer = nn.Linear(9, 192)
tokens_192d_projected = projection_layer(tokens_9d)
print(f"✅ Simulated Projected Tokens Shape (192D): {tokens_192d_projected.shape}")

# ==== 3. VISUALIZE RINGS ====
# (This part remains the same)
ring_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 165, 0)]
image_with_rings = image_rgb.copy()
for idx, (r0, r1) in enumerate(tokenizer.rings):
    cv2.circle(image_with_rings, center=tokenizer.center, radius=r1, color=ring_colors[idx], thickness=1)
plt.figure(figsize=(5, 5))
plt.imshow(image_with_rings)
plt.title("Radial Tokenizer Rings Overlay")
plt.axis("off")
plt.show()

# ==== 4. SAVE ALL OUTPUTS FOR NEXT STEPS ====
torch.save(tokens_9d.cpu(), os.path.join(TOK9_DIR, f"{IMG_NAME}.pt"))
torch.save(tokens_192d_projected.cpu(), os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
torch.save(projection_layer.state_dict(), os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))

print(f"💾 Saved 9D raw tokens to: {os.path.join(TOK9_DIR, f'{IMG_NAME}.pt')}")
print(f"💾 Saved 192D projected tokens to: {os.path.join(TOK192_DIR, f'{IMG_NAME}.pt')}")
print(f"💾 Saved SIMULATED projection weights to: {os.path.join(WEIGHTS_DIR, f'{IMG_NAME}.pt')}")

In [ ]:
# ====== RADIAL POSITIONAL ENCODING TEST ======
import torch
import os
from radial_positional_encoding import RadialPositionEmbedding

# ==== CONFIG ====
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D_projected") # Use the new directory
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")
os.makedirs(ENCODED_DIR, exist_ok=True)

# ==== LOAD PROJECTED TOKENS ====
# Load the tokens that were projected to 192D in the previous cell
tokens_192d = torch.load(os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
print(f"✅ Loaded projected 192D tokens with shape: {tokens_192d.shape}")

# ==== RUN POSITIONAL ENCODER ====
# The encoder expects 192D tokens, which is what we are providing
encoder = RadialPositionEmbedding(num_rings=4, embed_dim=192)
encoded_tokens = encoder(tokens_192d)
torch.save(encoded_tokens.cpu(), os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))

print(f"✅ Encoded tokens shape: {encoded_tokens.shape}")
print(f"💾 Saved encoded tokens to: {os.path.join(ENCODED_DIR, f'encoded_{IMG_NAME}.pt')}")

In [ ]:
# ====== VALIDATE OUTPUT SHAPES, CONTENT & VISUALIZE ======
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os

# ==== CONFIG ====
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK9_DIR = os.path.join(OUTPUT_DIR, "tokens_9D")
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D_projected")
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights_test")

# ==== LOAD ALL FILES ====
tokens_9d = torch.load(os.path.join(TOK9_DIR, f"{IMG_NAME}.pt"))
tokens_192d_projected = torch.load(os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
encoded_tokens = torch.load(os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))
projection_weights = torch.load(os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))

# ==== SHAPE & DTYPE VALIDATION ====
assert tokens_9d.shape == (1, 4, 9), f"❌ 9D token shape mismatch: {tokens_9d.shape}"
assert tokens_192d_projected.shape == (1, 4, 192), f"❌ 192D projected token shape mismatch: {tokens_192d_projected.shape}"
assert encoded_tokens.shape == (1, 4, 192), f"❌ Encoded token shape mismatch: {encoded_tokens.shape}"
print("✅ Shape and dtype checks passed.")

# ==== PRINT TOKEN CONTENT ====
print("\n=== RAW 9D TOKENS (Mean, Std, Median for each ring) ===")
df_9d = pd.DataFrame(tokens_9d.squeeze(0).detach().numpy())
print(df_9d.to_string(index=True))

print("\n=== SIMULATED PROJECTION WEIGHTS (9 -> 192) ===")
for name, weights in projection_weights.items():
    print(f"\nLayer: {name}, Shape: {weights.shape}")

# ==== VISUALIZE GRAPHS ====

# Plot 1: Raw 9D Tokens
plt.figure(figsize=(10, 4))
plt.imshow(tokens_9d.squeeze(0).detach().numpy(), aspect='auto', cmap='viridis')
plt.title("Raw 9D Tokens Per Ring")
plt.xlabel("Feature (Mean_R, G, B, Std_R, G, B, Median_R, G, B)")
plt.ylabel("Ring Index")
plt.colorbar()
plt.tight_layout()
plt.show()

# Plot 2: Projected 192D Tokens
tokens_proj_np = tokens_192d_projected.squeeze(0).detach().numpy()
plt.figure(figsize=(10, 4))
for i in range(4):
    plt.plot(tokens_proj_np[i], label=f"Ring {i+1}")
plt.title("192D Projected Tokens Per Ring")
plt.xlabel("Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Plot 3: Positionally Encoded 192D Tokens
tokens_enc_np = encoded_tokens.squeeze(0).detach().numpy()
plt.figure(figsize=(10, 4))
for i in range(4):
    plt.plot(tokens_enc_np[i], label=f"Ring {i+1}")
plt.title("192D Positionally Encoded Tokens Per Ring")
plt.xlabel("Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ====== CELL 4: DISPLAY FULL CONTENT OF ALL SAVED .pt FILES ======
import torch
import pandas as pd
import os

# --- CONFIGURATION ---
# Ensure these paths match the directories used in the previous cells
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK9_DIR = os.path.join(OUTPUT_DIR, "tokens_9D")
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D_projected")
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights_test")

# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000) # Increased width for better formatting
pd.set_option('display.precision', 6) # Display more decimal places

# --- 1. LOAD AND DISPLAY 9D RAW TOKENS ---
print("="*80)
print("1. Contents of the 9D Raw Tokens file (tokens_9d.pt)")
print("="*80)
tokens_9d = torch.load(os.path.join(TOK9_DIR, f"{IMG_NAME}.pt"))
print(f"Tensor Shape: {tokens_9d.shape}\n")
df_9d = pd.DataFrame(tokens_9d.squeeze(0).detach().numpy())
df_9d.columns = [f'Feat_{i}' for i in range(9)]
df_9d.index = [f'Ring_{i+1}' for i in range(4)]
print("Table of 9D Tokens (Rows are rings, Columns are features):")
print(df_9d)
print("\n\n")


# --- 2. LOAD AND DISPLAY PROJECTION WEIGHTS ---
print("="*80)
print("2. Contents of the Projection Weights file (projection_weights_test.pt)")
print("="*80)
projection_weights = torch.load(os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))
for name, weights in projection_weights.items():
    print(f"Layer Name: '{name}'")
    print(f"Tensor Shape: {weights.shape}")
    print("Tensor Values:")
    print(weights.detach().numpy())
    print("-" * 20)
print("\n\n")


# --- 3. LOAD AND DISPLAY 192D PROJECTED TOKENS ---
print("="*80)
print("3. Contents of the 192D Projected Tokens file (tokens_192d_projected.pt)")
print("="*80)
tokens_192d_projected = torch.load(os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
print(f"Tensor Shape: {tokens_192d_projected.shape}\n")
df_192d = pd.DataFrame(tokens_192d_projected.squeeze(0).detach().numpy())
df_192d.index = [f'Ring_{i+1}' for i in range(4)]
print("Table of 192D Projected Tokens:")
# The .iloc slicing has been REMOVED to show all columns
print(df_192d)
print("\n\n")


# --- 4. LOAD AND DISPLAY 192D ENCODED TOKENS ---
print("="*80)
print("4. Contents of the 192D Positionally Encoded Tokens file (encoded_{IMG_NAME}.pt)")
print("="*80)
encoded_tokens = torch.load(os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))
print(f"Tensor Shape: {encoded_tokens.shape}\n")
df_enc = pd.DataFrame(encoded_tokens.squeeze(0).detach().numpy())
df_enc.index = [f'Ring_{i+1}' for i in range(4)]
print("Table of 192D Encoded Tokens:")
# The .iloc slicing has been REMOVED to show all columns
print(df_enc)
print("\n\n")